In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
build_df = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
sample = pd.read_csv("../input/ashrae-energy-prediction/sample_submission.csv")
test_df = pd.read_csv("../input/ashrae-energy-prediction/test.csv")
train_df = pd.read_csv("../input/ashrae-energy-prediction/train.csv")
weather_test_df = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
weather_train_df = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")

In [2]:
np.set_printoptions(suppress=True)
pd.set_option('precision', 5) #设置精度
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [3]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)
weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
build_df = reduce_mem_usage(build_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [5]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])    
build_df['primary_use'] = build_df['primary_use'].astype('category')

In [6]:
train_df=train_df.merge(build_df,on='building_id',how='left')
test_df=test_df.merge(build_df,on='building_id',how='left')
train_df=train_df.merge(weather_train_df,on=['site_id','timestamp'],how='left')
test_df=test_df.merge(weather_test_df,on=['site_id','timestamp'],how='left')


In [7]:
del build_df,weather_train_df,weather_test_df
gc.collect()

11

In [8]:
train_df['primary_use'].replace({"Healthcare":"Other","Parking":"Other","Warehouse/storage":"Other","Manufacturing/industrial":"Other",
                                "Retail":"Other","Services":"Other","Technology/science":"Other","Food sales and service":"Other",
                                "Utility":"Other","Religious worship":"Other"},inplace=True)
test_df['primary_use'].replace({"Healthcare":"Other","Parking":"Other","Warehouse/storage":"Other","Manufacturing/industrial":"Other",
                                "Retail":"Other","Services":"Other","Technology/science":"Other","Food sales and service":"Other",
                                "Utility":"Other","Religious worship":"Other"},inplace=True)


In [9]:
for df in [train_df, test_df]:
    df['Month'] = df['timestamp'].dt.month.astype("uint8")
    df['DayOfMonth'] = df['timestamp'].dt.day.astype("uint8")
    df['DayOfWeek'] = df['timestamp'].dt.dayofweek.astype("uint8")
    df['Hour'] = df['timestamp'].dt.hour.astype("uint8")
    df['weekend'] = (df.timestamp.dt.weekday >=5).astype(int)

    

In [10]:
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                "2019-01-01"]
for df in [train_df, test_df]:
    df["Holiday"] = (df.timestamp.isin(holidays)).astype(int)

In [11]:
high=[0,2,3,4,8,9,14]
train_df['high_site']=train_df.site_id.isin(high).astype(int)
test_df['high_site']=test_df.site_id.isin(high).astype(int)


In [12]:
train_df['year_built'].fillna(-999,inplace=True)
test_df['year_built'].fillna(-999,inplace=True)
train_df.drop('floor_count',axis=1,inplace=True)
test_df.drop('floor_count',axis=1,inplace=True)

In [13]:
cols = ['air_temperature','cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed']
for col in cols:
    mean=train_df[col].notnull().mean()
    train_df[col].fillna(mean,inplace=True)
for col in cols:
    mean=test_df[col].notnull().mean()
    test_df[col].fillna(mean,inplace=True)   


In [14]:
timestamp_to_drop = list((train_df[(train_df['site_id'] == 0) & (train_df['timestamp'] < "2016-05-21 00:00:00")]).index)
print (len(timestamp_to_drop))
train_df.drop(timestamp_to_drop,axis='rows',inplace=True)

392857


In [15]:
electric_to_drop = list(train_df[(train_df['meter'] == 0) & (train_df['meter_reading'] == 0)].index)
print(len(electric_to_drop))
train_df.drop(electric_to_drop,axis='rows',inplace=True)

185592


In [16]:
bid1099_to_drop = list((train_df[(train_df['building_id']==1099)&(train_df['meter_reading'] > 16739)&(train_df['meter'] ==2)]).index)
print (len(bid1099_to_drop))
train_df.drop(bid1099_to_drop,axis='rows',inplace=True)

3395


In [17]:
le = LabelEncoder()
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype('uint8')
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype('uint8')
train_df['square_feet'] = np.log(train_df['square_feet'])
test_df['square_feet'] = np.log(test_df['square_feet'])

train_df['meter_reading'] = np.log1p(train_df['meter_reading'])


In [18]:
categorical_cols = ['building_id','site_id','meter','primary_use','Month','DayOfMonth','DayOfWeek','Hour','Holiday','high_site','weekend']

In [19]:
train_df.drop('timestamp',axis=1,inplace=True)
#train_df.drop('site_id',axis=1,inplace=True)
test_df.drop('timestamp',axis=1,inplace=True)
#test_df.drop('site_id',axis=1,inplace=True)
test_df.drop('row_id',axis=1,inplace=True)

In [20]:
y=train_df['meter_reading']
x=train_df.drop('meter_reading',axis=1)


In [21]:
del timestamp_to_drop,electric_to_drop,bid1099_to_drop
gc.collect()

74

In [22]:
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)
train_data = lgb.Dataset(x_train,y_train,categorical_feature=categorical_cols)
test_data = lgb.Dataset(x_test,y_test,categorical_feature=categorical_cols)


params = {'feature_fraction': 0.9,
          'objective': 'regression',
          'num_leaves':45,
          'learning_rate': 0.02,
          "boosting_type": "gbdt",
          "metric": 'rmse',
          'subsample': 0.9,
         }
print('************')
gbm = lgb.train(params,train_data,num_boost_round=5000,valid_sets=[train_data,test_data],early_stopping_rounds=100,verbose_eval=100)

************


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.20483	valid_1's rmse: 1.20436
[200]	training's rmse: 1.00241	valid_1's rmse: 1.00252
[300]	training's rmse: 0.912946	valid_1's rmse: 0.913197
[400]	training's rmse: 0.86045	valid_1's rmse: 0.860824
[500]	training's rmse: 0.83166	valid_1's rmse: 0.832244
[600]	training's rmse: 0.815212	valid_1's rmse: 0.815965
[700]	training's rmse: 0.80336	valid_1's rmse: 0.804312
[800]	training's rmse: 0.792546	valid_1's rmse: 0.793677
[900]	training's rmse: 0.783152	valid_1's rmse: 0.784454
[1000]	training's rmse: 0.775654	valid_1's rmse: 0.777056
[1100]	training's rmse: 0.76947	valid_1's rmse: 0.770998
[1200]	training's rmse: 0.762433	valid_1's rmse: 0.764091
[1300]	training's rmse: 0.756397	valid_1's rmse: 0.758183
[1400]	training's rmse: 0.750414	valid_1's rmse: 0.752379
[1500]	training's rmse: 0.745667	valid_1's rmse: 0.747789
[1600]	training's rmse: 0.740596	valid_1's rmse: 0.742826
[1700]	training's rmse: 0.7

In [23]:
%%time
predictions = []
step = 100000
for i in range(0, len(test_df), step):
    print(i)
    predictions.extend(np.expm1(gbm.predict(test_df.iloc[i: min(i+step, len(test_df)), :], num_iteration=gbm.best_iteration)))

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [24]:
%%time
sample['meter_reading'] = predictions
sample['meter_reading']=sample['meter_reading'].apply(lambda x:0 if x<0 else x)
#sample['meter_reading'].clip(lower=0,upper=None,inplace=True)
#sample.to_csv("second.csv",index='row_id')

CPU times: user 30.9 s, sys: 4.22 s, total: 35.1 s
Wall time: 34.6 s


In [25]:
%%time
sample.to_csv("sample33.csv",index=False,float_format='%.4f')


CPU times: user 4min 31s, sys: 2.54 s, total: 4min 33s
Wall time: 4min 36s
